
---

# 🔐 5. **Authentication & Authorization**

FastAPI supports powerful, secure, and modern auth mechanisms — perfect for production-ready APIs.
This section teaches you to implement **Login, Tokens, API keys**, and **role-based protection** easily. ✅

---

## 🔑 5.1 OAuth2 Password Flow

🔹 Use FastAPI's built-in support for **OAuth2** login flow — typically used for "username/password" based login.

### ⚙️ Example: Token-based login

```python
from fastapi import Depends, HTTPException, status, Form
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm

app = FastAPI()
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

@app.post("/token")
def login(form_data: OAuth2PasswordRequestForm = Depends()):
    if form_data.username == "admin" and form_data.password == "1234":
        return {"access_token": "secrettoken", "token_type": "bearer"}
    raise HTTPException(status_code=401, detail="Invalid credentials")

@app.get("/profile")
def read_profile(token: str = Depends(oauth2_scheme)):
    return {"token_received": token}
```

📌 `OAuth2PasswordRequestForm` reads `form-data` with `username` and `password`.

---

## 🔐 5.2 JWT Tokens with `pyjwt`

🔹 JWT (JSON Web Token) is the **industry standard** for secure, stateless authentication.

### 🔧 Install

```bash
pip install pyjwt
```

### 📄 Encode & Decode JWT

```python
import jwt
from datetime import datetime, timedelta

SECRET = "mysecret"

def create_token(data: dict):
    expire = datetime.utcnow() + timedelta(minutes=30)
    data.update({"exp": expire})
    return jwt.encode(data, SECRET, algorithm="HS256")

def decode_token(token: str):
    try:
        return jwt.decode(token, SECRET, algorithms=["HS256"])
    except jwt.ExpiredSignatureError:
        raise HTTPException(status_code=401, detail="Token expired")
```

📌 Use `create_token()` during login, and `decode_token()` to protect secure routes.

---

## 🔐 5.3 API Key Auth (Header-Based)

🔹 Common for **internal tools** or **microservices** — use simple headers like `X-API-Key`.

```python
from fastapi import Header

API_KEY = "supersecretkey"

@app.get("/internal")
def internal_data(x_api_key: str = Header(...)):
    if x_api_key != API_KEY:
        raise HTTPException(status_code=403, detail="Unauthorized")
    return {"status": "Welcome internal user"}
```

📌 Secure, lightweight, and works great with Postman, internal dashboards, etc.

---

## 🧑‍⚖️ 5.4 Role-Based Access Control (RBAC)

🔹 Differentiate between **admin vs user**, or **read vs write** roles.

```python
def get_current_user(token: str = Depends(oauth2_scheme)):
    payload = decode_token(token)
    return payload.get("role")

@app.get("/admin")
def admin_panel(role: str = Depends(get_current_user)):
    if role != "admin":
        raise HTTPException(status_code=403, detail="Admins only")
    return {"status": "Welcome Admin"}
```

📌 You can include `role`, `permissions`, or `scopes` inside the JWT payload.

---

## ✅ Summary Table

| Feature                   | What It Does                        | Ideal Use         |
| ------------------------- | ----------------------------------- | ----------------- |
| `OAuth2PasswordBearer`    | Basic login with `/token` route     | Login endpoints   |
| JWT with `pyjwt`          | Encrypted tokens with expiration    | Stateless Auth    |
| API Key Auth              | Fast and simple header-based auth   | Internal tools    |
| Role-Based Access Control | Restrict routes based on user roles | Admin-only routes |

---

